## Import Libraries

In [2]:
import os
import warnings
import joblib
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas

hv.extension('bokeh')
pn.extension()
warnings.filterwarnings('ignore')

class ToxinPredictionDashboard:
    def __init__(self):
        self.models_loaded = self._load_models()
        
        # Feature ranges and descriptions remain the same
        self.feature_ranges = {
            'Q':          {'min': -333, 'max': 283, 'initial': 20},
            'Q_1m':       {'min': -300, 'max': 1020, 'initial': 90},
            'SpecCond.':  {'min': 56,  'max': 10834, 'initial': 2910},
            'OrgC':       {'min': 1.4,  'max': 5.5,  'initial': 2.4},
            'OrgN':       {'min': 0.0,  'max': 1.6,  'initial': 0.3},
            'PO4':        {'min': 0.0,  'max': 0.53,  'initial': 0.08},
            'WaterTemp':  {'min': 13,   'max': 26.6,   'initial': 22.63},
            'DO':         {'min': 6.6,    'max': 10.7,   'initial': 8.17},
            'pH':         {'min': 7.0,  'max': 8.8,  'initial': 7.96}
        }

        self.feature_descriptions = {
            'Q':          'Flow Rate (m³/s)',
            'Q_1m':       'Flow Rate 1 Month Ago (m³/s)',
            'SpecCond.':  'Specific Conductance (μS/cm)',
            'OrgC':       'Organic Carbon (mg/L)',
            'OrgN':       'Organic Nitrogen (mg/L)',
            'PO4':        'Phosphate (mg/L)',
            'WaterTemp':  'Water Temperature (°C)',
            'DO':         'Dissolved Oxygen (mg/L)',
            'pH':         'pH'
        }

        self.setup_components()
        self.create_layout_components()
        self.update_predictions(None)

    def _load_models(self):
        try:
            self.random_forest_model = joblib.load(r'.\Model\rf_pipeline.joblib')
            self.xgboost_model = joblib.load(r'.\Model\xgb_pipeline.joblib')
            return True
        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def setup_components(self):
        # Create sliders
        self.feature_sliders = {}
        for feature, range_info in self.feature_ranges.items():
            slider = pn.widgets.FloatSlider(
                name=self.feature_descriptions[feature],
                start=range_info['min'],
                end=range_info['max'],
                value=range_info['initial'],
                step=(range_info['max'] - range_info['min']) / 100,
                width=400
            )
            slider.param.watch(self.update_predictions, 'value')
            self.feature_sliders[feature] = slider

        # Create sensitivity selector
        self.sensitivity_feature_selector = pn.widgets.Select(
            name='Select Feature for Sensitivity Analysis',
            options=list(self.feature_descriptions.values()),
            value=list(self.feature_descriptions.values())[0],
            width=400
        )
        self.sensitivity_feature_selector.param.watch(
            self.update_sensitivity_analysis_plot, 'value'
        )

    def create_layout_components(self):
        # Initialize components with specific sizes
        self.logo = pn.pane.Image('Logo.png', width=600)
        self.map_image = pn.pane.Image('Map.png', width=500)
        self.map_description = pn.pane.Markdown("""
            Study area map demonstrating data collection locations within the Sacramento – San Joaquin Delta (Delta):
            (a) Selected fourteen (14) data collection locations where toxin-producing Microcystis and other 17 environmental variables' data were collected between 2014 and 2019 within Delta and 
            (b) Range of maximum values of toxin-producing Microcystis (cells/ml) throughout the Delta
            """, width=500)
        
        self.random_forest_markdown = pn.pane.Markdown("", width=400)
        self.xgboost_markdown = pn.pane.Markdown("", width=400)
        self.model_comparison_plot = pn.pane.HoloViews(width=400)
        self.sensitivity_analysis_plot = pn.pane.HoloViews(width=600)
        
        # Bold disclaimer
        self.disclaimer = pn.pane.Markdown("""
        ---
        **Disclaimer: This dashboard is still in beta.**
        
        **Thank you for evaluating the HAB Dashboard.**
        
        **If you have feedback, suggestions or questions, please contact:**
        - **Peyman Namadi (Peyman.Hosseinzadehnamadi@Water.ca.gov)**
        - **Gourab Saha (gourab.saha@water.ca.gov)**
        - **Kevin He (Kevin.He@Water.ca.gov)**
        """)

    def update_predictions(self, event):
        # Get current values and make predictions
        current_values = {f: s.value for f, s in self.feature_sliders.items()}
        feature_order = ["Q", "Q_1m", "SpecCond.", "OrgC", "OrgN", "PO4", "WaterTemp", "DO", "pH"]
        sample_df = pd.DataFrame([current_values])[feature_order]

        rf_label = self.random_forest_model.predict(sample_df)[0]
        rf_prob = self.random_forest_model.predict_proba(sample_df)[0][1]
        xgb_label = self.xgboost_model.predict(sample_df)[0]
        xgb_prob = self.xgboost_model.predict_proba(sample_df)[0][1]

        # Update prediction displays
        rf_result_text = "High (>4000)" if rf_label == 1 else "Low (≤4000)"
        xgb_result_text = "High (>4000)" if xgb_label == 1 else "Low (≤4000)"

        self.random_forest_markdown.object = f"""
        ### Random Forest Prediction
        **Result:** {rf_result_text} toxin production
        **Probability:** {rf_prob:.2%} chance of high toxin production
        """
        
        self.xgboost_markdown.object = f"""
        ### XGBoost Prediction
        **Result:** {xgb_result_text} toxin production
        **Probability:** {xgb_prob:.2%} chance of high toxin production
        """

        # Update comparison plot
        comparison_df = pd.DataFrame({
            'Model': ['Random Forest', 'XGBoost'],
            'Probability': [rf_prob, xgb_prob]
        })

        self.model_comparison_plot.object = comparison_df.hvplot.bar(
            x='Model',
            y='Probability',
            title='Model Probability Comparison',
            width=350,  # Smaller width for thinner bars
            height=300,
            color=['#1f77b4', '#ff7f0e']
        ).opts(
            tools=['hover'],
            yformatter='%.2f',
            toolbar=None,
            fontsize={'title': 12, 'labels': 10, 'xticks': 10, 'yticks': 10},
            bar_width=0.3
        )
        
        self.update_sensitivity_analysis_plot(None)

    def update_sensitivity_analysis_plot(self, event):
        # Same implementation as before, but ensure the plot size matches the layout
        current_values = {f: s.value for f, s in self.feature_sliders.items()}
        selected_feature_desc = self.sensitivity_feature_selector.value
        selected_feature = next(k for k, v in self.feature_descriptions.items() if v == selected_feature_desc)
        range_info = self.feature_ranges[selected_feature]

        x_values = np.linspace(range_info['min'], range_info['max'], 20)
        rf_probs = []
        xgb_probs = []

        feature_order = ["Q", "Q_1m", "SpecCond.", "OrgC", "OrgN", "PO4", "WaterTemp", "DO", "pH"]
        for x_val in x_values:
            temp_values = current_values.copy()
            temp_values[selected_feature] = x_val
            temp_df = pd.DataFrame([temp_values])[feature_order]

            rf_probs.append(self.random_forest_model.predict_proba(temp_df)[0][1])
            xgb_probs.append(self.xgboost_model.predict_proba(temp_df)[0][1])

        sensitivity_df = pd.DataFrame({
            'Value': x_values,
            'Random Forest': rf_probs,
            'XGBoost': xgb_probs
        })

        self.sensitivity_analysis_plot.object = sensitivity_df.hvplot.line(
            x='Value',
            y=['Random Forest', 'XGBoost'],
            title=f'Sensitivity Analysis: {selected_feature_desc}',
            width=700,  # Increased width for better legend visibility
            height=400,
            xlabel=selected_feature_desc,
            ylabel='Probability of High Toxin Production (>4000)'
        ).opts(
            legend_position='top_right',
            tools=['hover'],
            yformatter='%.2f',
            toolbar=None,
            fontsize={'title': 12, 'labels': 10, 'xticks': 10, 'yticks': 10}
        )

    def create_dashboard_layout(self):
        # Headers with consistent styling
        input_header = pn.pane.Markdown("## Input Parameters", css_classes=['section-header'])
        results_header = pn.pane.Markdown("## Prediction Results", css_classes=['section-header'])
        comparison_header = pn.pane.Markdown("## Model Comparison", css_classes=['section-header'])
        sensitivity_header = pn.pane.Markdown("## Sensitivity Analysis", css_classes=['section-header'])

        # Apply CSS styles
        style = """
        .section-header {
            background-color: #f0f0f0;
            padding: 10px;
            margin-bottom: 10px;
        }
        """
        pn.extension(raw_css=[style])

        # Left side layout: inputs and sensitivity analysis
        left_col = pn.Column(
            input_header,
            *[self.feature_sliders[feat] for feat in self.feature_ranges.keys()],
            sensitivity_header,
            self.sensitivity_feature_selector,
            self.sensitivity_analysis_plot,
            width=700,  # Increased to match sensitivity plot width
            scroll=True
        )

        # Right side layout with map next to predictions
        model_results = pn.Column(
            results_header,
            pn.Row(
                pn.Column(
                    self.random_forest_markdown,
                    self.xgboost_markdown,
                    width=400
                ),
                pn.Column(
                    self.map_image,
                    self.map_description,
                    width=500
                )
            ),
            comparison_header,
            self.model_comparison_plot,
            width=900
        )

        # Main layout
        layout = pn.Column(
            pn.Row(self.logo, sizing_mode='stretch_width'),
            pn.Row(
                left_col,
                model_results
            ),
            self.disclaimer,
            sizing_mode='stretch_width'
        )

        return layout

    def serve_dashboard(self):
        return self.create_dashboard_layout().servable()

if __name__ == "__main__":
    dashboard_app = ToxinPredictionDashboard()
    if dashboard_app.models_loaded:
        try:
            port = 5006
            pn.serve(dashboard_app.create_dashboard_layout(), show=True, port=port)
        except Exception as e:
            print(f"Error serving dashboard: {str(e)}")
    else:
        print("Dashboard cannot be started due to model loading errors.")

Error serving dashboard: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted
